In [1]:
# packages for data handling
import numpy as np
import pandas as pd
import statistics as st
import math

# OpenTISim package
from opentisim import container_objects
from opentisim import container_defaults
from opentisim import container_system_offshore_barge

# Jupyter Widgets
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display
from ipywidgets import HBox, VBox

# packages for figures
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

import seaborn as sns
sns.set(style="ticks")

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Pretty-print a Python object to a stream
from pprint import pprint

# Scroll to specific cell
from jupyter_helpers.utilities import scroll_to_current_cell

# no warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
design_container_ship = 12500

In [3]:
def Port_System_Simulate(obj):
    
    # get startyear and lifecucle from widgets
    startyear = wdg_startyear.value
    lifecycle = wdg_lifecycle.value

    # 'scenario_data': demand for containers during 'years'
    years = list(range(startyear, startyear + lifecycle))
    demand = []
    for year in years:
        demand.append(wdg_throughput.value)
    scenario_data={'year': years, 'volume': demand}

    # instantiate Commodity objects, the inputs for the Commodity class
    container_defaults.container_data['new_panamax_perc'] = 100
    
    container = container_objects.Commodity(**container_defaults.container_data)
    container.scenario_data = pd.DataFrame(data=scenario_data)
    demand = [container]

    # instantiate vessels
    container_defaults.panamax_data["LOA"] = 230
    container_defaults.panamax_data["call_size"] = 2_000
    
    fully_cellular = container_objects.Vessel(**container_defaults.fully_cellular_data)
    panamax = container_objects.Vessel(**container_defaults.panamax_data)
    panamax_max = container_objects.Vessel(**container_defaults.panamax_max_data)
    post_panamax_I = container_objects.Vessel(**container_defaults.post_panamax_I_data)
    post_panamax_II = container_objects.Vessel(**container_defaults.post_panamax_II_data)
    new_panamax = container_objects.Vessel(**container_defaults.new_panamax_data)
    VLCS = container_objects.Vessel(**container_defaults.VLCS_data)
    ULCS = container_objects.Vessel(**container_defaults.ULCS_data)

    vessels = [fully_cellular, panamax, panamax_max, post_panamax_I, post_panamax_II, new_panamax, VLCS, ULCS] 

    # instantiate terminal object and simulate
    
    OffshoreBarge = container_system_offshore_barge.System(
        startyear = startyear,
        lifecycle = lifecycle,
        stack_equipment=wdg_stack_equipment.value,               
        laden_stack=wdg_stack_equipment.value,
        elements = demand + vessels,
        crane_type_defaults = container_defaults.sts_crane_data,
        offshore_distance = offshore_distance,
        barge_type = 'medium',
        bathymetry_factor = 0.80,
        allowable_berth_occupancy = 0.70,
        laden_perc=0.80, 
        reefer_perc=0.10,
        empty_perc=0.05, 
        oog_perc=0.05,
        transhipment_ratio = 0.0,
        debug = True)
    OffshoreBarge.simulate()

    # display output
    if wdg_chkbx_output_0.value:
        container.plot_demand()

    self = OffshoreBarge
    quay = self.find_elements(self, Quay_wall)

    print('')
    print('Number of quays: {}'.format(
        len(self.find_elements(self, Quay_wall))))
    print('Quay length: {:.2f}'.format(
        np.sum([element.length for element in self.find_elements(self, Quay_wall)])))
    print('Quay unit rate: {:.2f} USD / m'.format(
        np.mean([element.unit_rate for element in self.find_elements(self, Quay_wall)])))
    print('Quay retaining height: {:.2f} m [2 * (retaining_height + freeboard)]'.format(
        np.mean([element.retaining_height for element in 
                 self.find_elements(self, Quay_wall)])))
    print('Apron surface: {:.2f} ha'.format(
        np.sum([element.length for element in 
                self.find_elements(self, Quay_wall)])*quay[0].apron_width*0.0001))
    print('STS cranes: {}'.format(
        len(self.find_elements(self, Cyclic_Unloader))))
    print('Horizontal transport: {} (coupled with STS cranes)'.format(
        len(self.find_elements(self, Horizontal_Transport))))
    print('Stack equipment: {} (type: {})'.format(
        len(self.find_elements(self, Stack_Equipment)), self.stack_equipment))
    print('Number of laden stacks: {}'.format(
        len(self.find_elements(self, Laden_Stack))))
    print('Empty handlers: {} (coupled with STS cranes)'.format(
        len(self.find_elements(self, Empty_Handler))))
    print('Number of gates: {}'.format(
        len(self.find_elements(self, Gate))))

    if wdg_chkbx_output_1.value:
        self.terminal_elements_plot(demand_step=100000)

    if wdg_chkbx_output_2.value:
        cash_flows, cash_flows_WACC_real = self.add_cashflow_elements(OffshoreBarge, 
                                                                      container_objects.Labour(**container_defaults.labour_data))
        plot.cashflow_plot(OffshoreBarge, cash_flows, title='Cash flow plot')
        plot.cashflow_plot(OffshoreBarge, cash_flows_WACC_real, title='Cash flow plot (WACC real)')

    if wdg_chkbx_output_3.value:
        OffshoreBarge.land_use_plot()

    if wdg_chkbx_output_4.value:
        df = self.NPV(OffshoreBarge, container_objects.Labour(**container_defaults.labour_data))
        print(df)

In [4]:
wdg_startyear = widgets.IntSlider(min=2000, max=2050, step=1, description='Start year:', value=2020)
wdg_lifecycle = widgets.IntSlider(min=5, max=50, step=1, description='Life cycle:', value=10)
wdg_throughput = widgets.IntSlider(min=0, max=2_000_000, step=1, description='Throughput:', value=1_000_000)
wdg_postpanamax = widgets.IntSlider(min=0, max=100, step=1, description='Post Panamax I percentage:', value=0)
wdg_newpanamax = widgets.IntSlider(min=0, max=100, step=1, description='New-Panamax percentage:', value=100)
wdg_ulcs = widgets.IntSlider(min=0, max=100, step=1, description='ULCS percentage:', value=0)
wdg_start = widgets.Button(description='Simulate')
wdg_chkbx_output_0 = widgets.Checkbox(description='Demand', value=False)
wdg_chkbx_output_1 = widgets.Checkbox(description='Terminal elements', value=False)
wdg_chkbx_output_2 = widgets.Checkbox(description='Cash flows', value=False)
wdg_chkbx_output_3 = widgets.Checkbox(description='Land use', value=False)
wdg_chkbx_output_4 = widgets.Checkbox(description='NPV', value=False)

wdg_stack_equipment = widgets.RadioButtons(
    options=['rtg', 'rmg', 'rs', 'sc'],
    description='Stack equipment:',
    disabled=False,
    layout={'width': 'max-content'}
)

wdg_start.on_click(Port_System_Simulate)

left_box = VBox([wdg_startyear, wdg_lifecycle, wdg_throughput])
middle_box = VBox([wdg_postpanamax, wdg_newpanamax, wdg_ulcs])
right_box = VBox([wdg_stack_equipment])
input_panel = HBox([left_box, middle_box, right_box, wdg_start])
output_panel = HBox([wdg_chkbx_output_1, wdg_chkbx_output_2, wdg_chkbx_output_3, wdg_chkbx_output_4])
VBox([input_panel, output_panel])
